In [70]:
# Step 0: Install required packages
!pip install requests transformers

In [84]:
%pip install -q -U "google-genai>=1.0.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 16.3 MB/s eta 0:00:00


In [1]:
# Install requests for API calls
!pip install requests


In [96]:
import os
import json
import requests
from google import genai

# Load API keys from Colab secrets
# GOOGLE_API_KEY is already set in environment variables
client = genai.Client()  # automatically uses GOOGLE_API_KEY from os.environ
MODEL_ID = "gemini-2.5-flash-preview-05-20"
SERPER_API_KEY = "64d6ae585c82b13dbf6085e4ffbbeb499fb4db4d"

In [97]:
# -----------------------------
# Step 1: Gemini API dream interpretation
# -----------------------------
def interpret_dream_gemini(user_query):
    """
    Sends the dream query to Gemini 2.5 Flash model and returns interpretation.
    """
    try:
        response = client.models.generate_content(
            model=MODEL_ID,
            contents=f"You are an expert dream interpreter. Explain the meaning of this dream:\n{user_query}"
        )
        return response.text
    except Exception as e:
        return f"Error: {e}"


In [98]:
def get_web_insights_serper(user_query, top_k=3):
    """
    Uses Serper API to get top-k related snippets for the dream query.
    """
    try:
        url = "https://google.serper.dev/search"
        headers = {"X-API-KEY": SERPER_API_KEY, "Content-Type": "application/json"}
        payload = {"q": f"{user_query} dream meaning"}
        response = requests.post(url, headers=headers, json=payload)
        data = response.json()

        # Try multiple possible keys for results
        results = data.get("organic", []) or data.get("organic_results", []) or data.get("items", [])
        snippets = []
        for item in results[:top_k]:
            # Some items may have "snippet" or "description"
            snippet = item.get("snippet") or item.get("description")
            if snippet:
                snippets.append(snippet)

        if not snippets:
            snippets.append("No web insights found or API returned empty results.")

        return snippets

    except Exception as e:
        return [f"Error fetching web insights: {e}"]


In [106]:
# -----------------------------
# Step 3: Merge interpretations
# -----------------------------
def merge_dream_interpretation(user_query):
    """
    Provide a brief, structured interpretation of the dream about [user_query]. Focus only on core symbols and their most likely meanings. Keep the 'ultimate_interpretation' short (3-4 sentences max), merging only the most relevant web insights if available. Prioritize clarity and brevity over depth. Example format:*
    """
    llm_response = interpret_dream_gemini(user_query)
    web_snippets = get_web_insights_serper(user_query)

    ultimate_interpretation = f"{llm_response}"
    if web_snippets:
        ultimate_interpretation += " Merged with key themes from web insights: " + "; ".join(web_snippets)

    json_output = {
        "dream_query": user_query,
        "ai_interpretation": llm_response,
        "web_insights": web_snippets,
        "ultimate_interpretation": ultimate_interpretation
    }

    try:
        print("JSON Output:\n", json.dumps(json_output, indent=2))
    except Exception as e:
        print(f"JSON Encode Error: {e}")

    return json_output

In [107]:
# -----------------------------
# Step 4: Example run
# -----------------------------
user_query = "I dreamt of flying over the ocean"
dream_result = merge_dream_interpretation(user_query)

JSON Output:
 {
  "dream_query": "I dreamt of flying over the ocean",
  "ai_interpretation": "Ah, this is a truly magnificent and rich dream! As an expert dream interpreter, I can tell you that \"flying over the ocean\" is a powerful archetypal image, laden with meaning about your inner state and outer life circumstances.\n\nLet's break down the key symbols:\n\n1.  **Flying:**\n    *   **Freedom and Liberation:** Flying in dreams almost universally signifies a desire for or an experience of freedom. You might be feeling liberated from constraints, responsibilities, or situations that previously weighed you down.\n    *   **Transcendence and Perspective:** It suggests an ability to rise above everyday concerns, gain a higher perspective, and see the \"bigger picture\" of your life. You're not bogged down by details but can observe from an elevated viewpoint.\n    *   **Mastery and Control:** Often, flying dreams indicate a feeling of being in control of your life, your destiny, and your